In [103]:
import math
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import KFold


In [138]:
train_in = np.loadtxt('./Datasets/train_in.csv', 
                  delimiter=',',  
                  skiprows=0,      
                  dtype=float)       
test_in = np.loadtxt('./Datasets/test_in.csv', 
                  delimiter=',',   
                  skiprows=0,      
                  dtype=float)       
train_out = np.loadtxt('./Datasets/train_out.csv', 
                  delimiter=',',  
                  skiprows=0,      
                  dtype=float)       
test_out = np.loadtxt('./Datasets/test_out.csv', 
                  delimiter=',',   
                  skiprows=0,      
                  dtype=float)       

In [139]:
X_train, X_val, y_train, y_val = train_test_split(train_in, train_out, train_size=0.8, random_state=0)
X_test = test_in
y_test = test_out
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

(1365, 256) (342, 256) (1365,) (342,)


In [140]:
# input dataset
def preprocessing(datasets):
    return np.hstack((datasets, np.ones(len(datasets)).reshape(len(datasets),1)))

In [141]:
def get_dummies(list, array):
    for i, each in zip(list, array):
        each[int(i)] = 1
    return array

In [167]:
# sigmoid function 
def sig(x):
    return (1/(1+np.exp(-x)))

def deridig(y):
    return y*(1-y)

In [143]:
Len_train = len(X_train)
Len_val = len(X_val)
Len_test = len(X_test)

X_train = preprocessing(X_train)
X_val = preprocessing(X_val)
X_test = preprocessing(X_test)

y_train_dummy = np.zeros(Len_train*10).reshape(Len_train, 10)
y_train_dummy = get_dummies(y_train, y_train_dummy)
y_val_dummy = np.zeros(Len_val*10).reshape(Len_val, 10)
y_val_dummy = get_dummies(y_val, y_val_dummy)
y_test_dummy = np.zeros(Len_test*10).reshape(Len_test, 10)
y_test_dummy = get_dummies(y_test, y_test_dummy)

In [144]:
kf = KFold(n_splits=5)

kfold_index_list = []

for cv_index_train, cv_index_test in kf.split(X_train):
    kfold_index_list.append((cv_index_train, cv_index_test))

In [153]:
def training(lr, w, train_x, y_train_dummy):
    for iter in range(100000):
        y_hat = sig(np.dot(train_x, w))
        # e = pow((y_train_dummy - y) * (y_train_dummy - y), 0.5)
        diff = y_train_dummy - y_hat
        # if iter % 100 == 0 :
        #    print(sum(sum(abs(e))))
        if sum(sum(abs(diff))) <= 1e-04:
            # print(iter)
            break
        # updata w0(weight)
        # print(w0.shape, train_x.shape, e.shape)
        # print((1-w0) * w0)
        # w0 += lr * np.dot(train_x.T, e) * ((1-w0) * w0)
        w += lr * np.dot(train_x.T, diff)
        # print(sum(sum(((1-w0) * w0))))
    return w

In [146]:
def predict(lr, w, train_x, y_train_dummy):
    y_hat = sig(np.dot(train_x, w))
    return np.argmax(y_hat, axis=1)

In [163]:
np.random.seed(2022)
w0 = np.random.random((257,10))
lr = 0.3
# w = train(lr, w0, train_x, y_train_dummy)
results = []
w_list = []

for i in range(5):
    w = training(lr, w0, X_train[kfold_index_list[i][0]], y_train_dummy[kfold_index_list[i][0]])

    pred_train = predict(lr, w, X_train[kfold_index_list[i][1]], y_train_dummy[kfold_index_list[i][1]])
    acc_train = accuracy_score(y_train[kfold_index_list[i][1]], pred_train)

    pred_val = predict(lr, w, X_val, y_val_dummy)
    acc_val = accuracy_score(y_val, pred_val)

    pred_test = predict(lr, w, X_test, y_test_dummy)
    acc_test = accuracy_score(y_test, pred_test)

    w_list.append(w)
    results.append((acc_train, acc_val, acc_test))


<ipython-input-158-62129ee6f76d>:3: RuntimeWarning: overflow encountered in exp
  return (1/(1+np.exp(-x)))
<ipython-input-158-62129ee6f76d>:3: RuntimeWarning: overflow encountered in exp
  return (1/(1+np.exp(-x)))
<ipython-input-158-62129ee6f76d>:3: RuntimeWarning: overflow encountered in exp
  return (1/(1+np.exp(-x)))
<ipython-input-158-62129ee6f76d>:3: RuntimeWarning: overflow encountered in exp
  return (1/(1+np.exp(-x)))
<ipython-input-158-62129ee6f76d>:3: RuntimeWarning: overflow encountered in exp
  return (1/(1+np.exp(-x)))
<ipython-input-158-62129ee6f76d>:3: RuntimeWarning: overflow encountered in exp
  return (1/(1+np.exp(-x)))
<ipython-input-158-62129ee6f76d>:3: RuntimeWarning: overflow encountered in exp
  return (1/(1+np.exp(-x)))
<ipython-input-158-62129ee6f76d>:3: RuntimeWarning: overflow encountered in exp
  return (1/(1+np.exp(-x)))
<ipython-input-158-62129ee6f76d>:3: RuntimeWarning: overflow encountered in exp
  return (1/(1+np.exp(-x)))
<ipython-input-158-62129ee6f

In [164]:
for acc, acc_val, acc_test in results:
    print("Accuracy train: ", acc, "Accuracy validation", acc_val, "Accuracy test", acc_test)

Accuracy train:  0.8974358974358975 Accuracy validation 0.9385964912280702 Accuracy test 0.836
Accuracy train:  0.9413919413919414 Accuracy validation 0.9415204678362573 Accuracy test 0.855
Accuracy train:  0.989010989010989 Accuracy validation 0.9415204678362573 Accuracy test 0.852
Accuracy train:  1.0 Accuracy validation 0.9385964912280702 Accuracy test 0.854
Accuracy train:  1.0 Accuracy validation 0.9385964912280702 Accuracy test 0.854


In [166]:
## confusion Test
confusion_matrix(predict(lr, w_list[1], X_test, y_test_dummy), y_test)

<ipython-input-158-62129ee6f76d>:3: RuntimeWarning: overflow encountered in exp
  return (1/(1+np.exp(-x)))


array([[213,   0,   5,   4,   2,   4,   5,   0,   7,   0],
       [  0, 117,   1,   0,   1,   0,   0,   1,   0,   1],
       [  6,   2,  85,   3,   6,   0,   6,   5,   3,   1],
       [  0,   0,   1,  63,   0,   5,   0,   2,   3,   0],
       [  3,   1,   2,   0,  68,   1,   2,   3,   2,   5],
       [  0,   0,   1,   2,   2,  42,   5,   0,   5,   0],
       [  0,   0,   2,   0,   1,   0,  72,   0,   0,   0],
       [  0,   1,   1,   1,   4,   1,   0,  51,   1,   5],
       [  1,   0,   3,   4,   0,   1,   0,   0,  70,   3],
       [  1,   0,   0,   2,   2,   1,   0,   2,   1,  73]], dtype=int64)

In [152]:
# w0 = np.random.random((257,10))
# lr = 0.3
# w = training(lr, w0, X_train, y_train_dummy)